In [2]:
import numpy as np
import copy
import itertools
import random
import time
#import math

In [3]:
def tttshow(ox):
    '''显示棋盘，参数是棋盘矩阵'''
    #'''打印列标号'''
    print("{0:^5}".format('x\y'), end='') # 5个字符
    for j in range(ox.shape[1]):
        print("{0:^5}".format(j), end='') # 5个字符          
    print('\n')  # 换行,打印空行        
      
    #'''打印棋盘'''            
    for i in range(ox.shape[0]):        #行数
        #'''打印行标号'''
        print("{0:^5}".format(i), end='') # 5个字符
        #'''打印该行棋盘'''    
        for j in range(ox.shape[1]):
            if ox[i][j] == 0:    #未落子  5个字符
                print("{0:^5}".format('~'), end='')
            elif ox[i][j] == 1:   # x 落子  5个字符
                print("{0:^5}".format('X'), end='')
            elif ox[i][j] == -1:  # o 落子
                print("{0:^5}".format('Q'), end='') 
        print('\n')  
                

def ttt_winner(ox):
    '''计算方阵连线情况,1代表'x'连成一线，-1代表'o'连成一线，0代表没有'''
    '''把各行列的和组成集合，查找集合中是否存在3或-3，代表连成一行'''
    ox_line = set(ox.sum(axis=1)) | set(ox.sum(axis=0))
    '''第一条对角线的和加入列表'''
    ox_line.add(sum([ox[i, i] for i in range(ox.shape[0]) ]))
    '''第二条对角线的和加入列表'''
    ox_line.add(sum([ox[ox.shape[0]-1-i,i] for i in range(ox.shape[0]) ]))
    
    return -1 if -ox.shape[0] in ox_line else 1 if ox.shape[0] in ox_line else 0
    
    
def ttt_simu(ttt, playturn='o', ms=[]):
    '''根据给定棋盘现状和后续下法，返回棋盘最终状态，谁赢或者平局'''
    ox = copy.deepcopy(ttt)
    for i in range(len(ms)):
        ox[ms[i]] = 1 if playturn=='x' else -1     # 更新棋盘   
        if ttt_winner(ox) == 1:
            return ttt_winner(ox) + (len(ms)-i-1)
        elif ttt_winner(ox) == -1:
            return ttt_winner(ox) - (len(ms)-i-1)       
        elif ttt_winner(ox) == 0 :
            playturn = 'o' if playturn=='x' else 'x'
    return 0
    

def ai_montecarlo_search(ox, playturn='o', ox_availables=set(), mounts=1000):
    timebegin = time.time()
    move_value = {x:0 for x in ox_availables}
    for count in range(mounts):
        mv = []
        a_setlist = list(ox_availables)
        ms = len(a_setlist)
        for i in range(ms):
            mv.append(a_setlist.pop(random.randint(0,len(a_setlist)-1)))
        move_value[mv[0]] += ttt_simu(ox, playturn, mv)
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==maxvalue ]
    #print(move_value)
    #print(mvlist)
    print(time.time()-timebegin)
    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)
    
    
def ai_full_search(ox, playturn='o', ox_availables=set()):
    '''AI走一步，但每走一步都穷尽所有走法，找到最好的,'''
    timebegin = time.time()
    '''生成剩余下法的胜利初始化表'''
    move_value = {x:0 for x in ox_availables}
    #'''一个产生下法序列的生成器函数，穷举直到不再有新的下法序列'''
    #'''使用了排列生成，生成函数是个函数生成器； 返回剩余下法的所有排列'''
    mvs = itertools.permutations(ox_availables) 
    for m in mvs:
        move_value[m[0]] += ttt_simu(ox, playturn, m)
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==maxvalue ]
    #print(move_value)
    #print(mvlist)
    print(time.time()-timebegin)
    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)

          
def update(ox, player, location):
    '''# player在location(x,y)处落子，更新棋盘，,更新可用位置，计算此时棋局胜负'''
    ox[location] = 1 if player=='x' else -1
    global ttt_availables 
    ttt_availables.remove( location )


In [4]:
''' # 3 x 3 矩阵，值是棋子类型，0空，1代表'x'，-1代表'o'; ''' 
ttt = np.zeros((3, 3), dtype=int )
ttt_availables = set( [(i,j) for i in range(3) for j in range(3)] )  
player = 'x'

In [5]:
while True :

    tttshow(ttt)

    if player == 'x':
        #print('请选手选择落子处的数字，如23代表在第2行第3列落子：', end='')
        #xy = input()
        #playlocation = (int(xy[0]), int(xy[1]) )
        playlocation = ai_montecarlo_search(ttt, player, ttt_availables)
        update(ttt, player, playlocation)
        if ttt_winner(ttt) == 1:
            print('选手 赢')
            tttshow(ttt)
            break
        if ttt_availables == set():
            print('平局')
            tttshow(ttt)
            break
        player = 'o'

    if player == 'o':
        playlocation = ai_full_search(ttt, player, ttt_availables)
        #playlocation = ai_montecarlo_search(ttt, player, ttt_availables)
        update(ttt, player, playlocation)
        if ttt_winner(ttt) == -1:
            print('计算机 赢')
            tttshow(ttt)
            break 
        if ttt_availables == set():
            print('平局')
            tttshow(ttt)
            break
        player = 'x'


 x\y   0    1    2  

  0    ~    ~    ~  

  1    ~    ~    ~  

  2    ~    ~    ~  

{(0, 1): 60, (1, 2): 59, (0, 0): 72, (2, 1): 13, (2, 0): 116, (1, 1): 205, (2, 2): 138, (1, 0): 51, (0, 2): 73}
[(1, 1)]
0.424602746963501
{(0, 1): 8136, (1, 2): 8136, (0, 0): 6192, (2, 1): 8136, (2, 0): 6192, (2, 2): 6192, (1, 0): 8136, (0, 2): 6192}
[(0, 0), (2, 0), (2, 2), (0, 2)]
12.770867347717285
 x\y   0    1    2  

  0    ~    ~    ~  

  1    ~    X    ~  

  2    Q    ~    ~  

{(0, 1): 157, (1, 2): 179, (0, 0): 256, (2, 1): 243, (2, 2): 239, (1, 0): 231, (0, 2): 13}
[(0, 0)]
0.304107666015625
{(0, 1): 240, (1, 2): 252, (2, 1): 128, (2, 2): -40, (1, 0): 276, (0, 2): 264}
[(2, 2)]
0.1496107578277588
 x\y   0    1    2  

  0    X    ~    ~  

  1    ~    X    ~  

  2    Q    ~    Q  

{(0, 1): -42, (1, 2): -70, (2, 1): 220, (1, 0): -205, (0, 2): -195}
[(2, 1)]
0.21784543991088867
{(0, 1): 2, (1, 2): 5, (1, 0): 8, (0, 2): 7}
[(0, 1)]
0.0
 x\y   0    1    2  

  0    X    Q    ~  

  1    ~